# Introduction to _pyspark_

Spark and its related packages are constantly changing, and even the most basic scripts may become unusable from version to version. Threfore it is a good idea to be familiar with the documentation part, which is (trying to be) updated and helpful. Here are some relevant documentation links:

* [Spark 2.0.2][spark]
    * General concepts
        * [Programing guide][pg]
        * [Data structures][ds] - this includes explanations about DataFrames, DataSets and SQL
    * Python API
        * [pyspark package][pyspark] - this includes the [SparkConf][conf], [SparkContext][sc] and [RDD][rdd] classes
        * [pyspark.sql module][sql] - this includes the [SparkSession][ss], [DataFrame][df], [Row][row] and [Column][col] classes

[spark]: https://spark.apache.org/docs/2.0.2/index.html "Spark 2.0.2"
[pg]: https://spark.apache.org/docs/2.0.2/programming-guide.html "Spark programming guide"
[ds]: https://spark.apache.org/docs/2.0.2/sql-programming-guide.html "Data structures programming guide"
[pyspark]: https://spark.apache.org/docs/2.0.2/api/python/index.html "pyspark"
[conf]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.html#pyspark.SparkConf "SparkConf"
[sc]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.html#pyspark.SparkContext "SparkContext"
[rdd]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.html#pyspark.RDD "RDD"
[sql]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html "pyspark.sql module"
[ss]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.SparkSession "SparkSession"
[df]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame "DataFrame"
[row]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.Row "Row"
[col]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.Column "Column"

## Example 1 - RDD fundamentals

Read the Moby Dick text and answer the questions below. 

> **NOTE:** See [here][dbfs] for details about uploading files to the DBFS.

[dbfs]: https://docs.databricks.com/user-guide/importing-data.html "DBFS uploading"

In [5]:
words = sc\
    .textFile("/FileStore/tables/yg6aazpx1490278902208/melville_moby_dick-7006e.txt")\
    .flatMap(lambda line: line.split())\
    .filter(lambda word: word.isalpha())\
    .map(lambda word: word.lower())
words.take(5)

### Question 1

How may words are in the book? (words contain only letters)

In [8]:
words.count()

### Question 2

How many _unique_ words are in the book?

In [11]:
unique_words = words.groupBy(lambda word: word)
unique_words.take(5)

In [12]:
unique_words.count()

### Question 3

What is the most common word in the book?

In [15]:
word_count = unique_words\
    .mapValues(lambda group: len(group))\
    .sortBy(lambda word_count: word_count[1], ascending=False)
print word_count.take(10)

### Question 4

What is the most common word in the book which is not a [stop-word][1]? (a file with the English stop-words is available in the folder)

[1]: https://en.wikipedia.org/wiki/Stop_words "Stop words - Wikipedia"

In [18]:
stop_words = sc\
    .textFile("/FileStore/tables/pp6ku2g01490279937078/english_stop_words-12e81.txt")\
    .map(lambda word: (word, 1))
print stop_words.take(10)

In [19]:
word_count_2 = word_count\
    .subtractByKey(stop_words)\
    .sortBy(lambda word_count: word_count[1], ascending=False)
print word_count_2.take(5)

> **Your turn 1:** Read the file "english words" into an RDD and answer the following questions:
> 1. How many words are listed in the file?
> 2. What is the most common first letter?
> 3. What is the longest word in the file?
> 4. How many words include all 5 vowels?

## Example 2 - DataFrames fundamentals

Each record in the "dessert" dataset describes a group visit at a restaurant. Read the data and answer the questions below.

In [23]:
dessert = spark.read.csv("/FileStore/tables/sdztx2671490282633198/dessert.csv", 
                         header=True, inferSchema=True)\
  .drop('id')\
  .withColumnRenamed('day.of.week', 'weekday')\
  .withColumnRenamed('num.of.guests', 'num_of_guests')\
  .withColumnRenamed('dessert', 'purchase')\
  .withColumnRenamed('hour', 'shift')
dessert.show(5)

In [24]:
dessert.printSchema()

The DataframeReader object used above is sometimes confusing, so I show below how to first load the data as an RDD, and then modify it into a dataFrame. During this process we also remove the header using a combination of _zipWithIndex()_ and _filter()_ (taken from [here][1]). By looking at the file we see the "schema", which is used by the second _map()_.

[1]: http://stackoverflow.com/a/31798247/3121900

In [26]:
dessert_rdd = sc\
    .textFile("/FileStore/tables/sdztx2671490282633198/dessert.csv")\
    .map(lambda line: line.split(','))\
    .zipWithIndex()\
    .filter(lambda tup: tup[1] > 0)\
    .map(lambda tup: [tup[0][1],           # weekday
                      int(tup[0][2]),      # num_of_guests
                      tup[0][3],           # shift
                      int(tup[0][4]),      # table
                      tup[0][5]=='TRUE'])  # purchase

columns = ['weekday', 'num_of_guests', 'shift', 'table', 'purchase']
dessert = spark.createDataFrame(dessert_rdd,
                                schema=columns)
dessert.show(5)

### Question 1

How many groups purchased a dessert?

In [29]:
col = dessert.purchase
dessert.where(col).count()

### Question 2

How many groups purchased a dessert on Mondays?

In [32]:
col = (dessert.weekday == 'Monday') & (dessert.purchase)
dessert.where(col).count()

### Question 3

How many _visitors_ purchased a dessert?

In [35]:
dessert\
    .where(dessert.purchase)\
    .agg({'num_of_guests': 'sum', 'table': 'mean'})\
    .show()

### Question 4

For each weekday - how many groups purchased a dessert?

In [38]:
dessert\
    .where(dessert.purchase)\
    .groupBy('weekday')\
    .agg({'shift': 'count', 'num_of_guests': 'sum'})\
    .show()

### Question 5

Add to _dessert_ a new column called 'no purchase' with the negative of 'purchse'.

In [41]:
dessert = dessert.withColumn('no_purchase', ~dessert.purchase)
dessert.show(5)

### Question 6

Create a pivot table showing how the purchases were influenced by the size of the group.

In [44]:
dessert.crosstab('num_of_guests', 'weekday').show()

> **Your turn 2:** Read the file "weights" into an Dataframe and answer the following questions:
> 1. Create a new Dataframe with the data of the males only and call it _males_.
> 2. How many males are in the table? What is the mean height and weight of the males?
> 3. What is the height of the tallest female who is older than 40?
> 4. Create a new Dataframe with two columns for the age and the average weight of the people in this age.